In [2]:
from __future__ import print_function
from datascience import *
from datetime import date, datetime, timedelta
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
from datascience import * # datascience has plotting features built in
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")
import warnings
warnings.filterwarnings('ignore')
from datascience import *
import numpy as np
import plotly.express as px
Table.interactive_plots()
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Pulling Data From Pantry Checkout Log

In [3]:

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# The ID and range of a sample spreadsheet.
SPREADSHEET_ID = '1waiCSO7hm8kmWLaG8OdB8_7ksfEbm_B3CyKHbXTIf5E'
RANGE_NAME = 'Fall22!A2:F'
dateColumn = []
timeColumn = []
itemBarcodeColumn = []
quantityColumn = []
itemNameColumn = []
remainingColumn = []
def getSheetInfo():
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('sheets', 'v4', credentials=creds)

        # Call the Sheets API
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                    range=RANGE_NAME).execute()
        values = result.get('values', [])

        if not values:
            print('No data found.')
            return
        for row in values:
            dateColumn.append(row[0])
            timeColumn.append(row[1])
            itemBarcodeColumn.append(row[2])
            quantityColumn.append(int(row[3]))
            itemNameColumn.append(row[4])
            remainingColumn.append(int(row[5]))
    except HttpError as err:
        print(err)

getSheetInfo()

In [4]:
checkout = Table().with_columns("Date", dateColumn,"Time", timeColumn, "Item Barcode", itemBarcodeColumn, "Quantity", quantityColumn, "Item Name", itemNameColumn, "Remaining #", remainingColumn)
checkout

Date,Time,Item Barcode,Quantity,Item Name,Remaining #
Thursday 8/18,12:15,027862020120,9,Firm Tofu,0
Thursday 8/18,12:16,027862020168,10,Soft Tofu,0
Thursday 8/18,12:17,029243050351,1,Macaroni,0
Monday 8/22,15:57,024182025064,1,Edensoy Soy Milk,47
,,apple,1,Apple,-1
Thursday 9/8,10:01,754686002437,1,"Almond Original, Unsweetened Almond Milk",35
,,027000500064,1,Hunt's Pasta Sauce,154
,,apple juice,1,"Totally Juice, 100% Apple Juice",33
,,035200056427,1,Brown Rice,177
Thursday 9/8,10:02,apple juice,1,"Totally Juice, 100% Apple Juice",32


## Filtering for Previous Week's Data Only

In [30]:
today = date.today()
startOfLastWeek = today - timedelta(days=today.weekday() + 14)
endOfLastWeek = startOfLastWeek + timedelta(days = 6)

print("Start of Last Week: " + str(startOfLastWeek))
print("End of Last Week: " + str(endOfLastWeek))

Start of Last Week: 2023-01-30
End of Last Week: 2023-02-05


Starting from the top of the spreadsheet, look for the row that has the first checkout from the first day of last week and the last checkout from the last day of last week.

In [31]:
dateArray = checkout.column("Date")
startIndex = -1
endIndex = len(dateArray)
numPeople = 0
for i in range(len(dateArray)):
    if (dateArray[i] != 'nan'  and 4 <= len(dateArray[i]) <= 14):
        z = dateArray[i].split(' ')
        if len(z) > 1: #kind of hackey but should change once we get a uniform thing for date's
            currDate = z[1]
            currDate = date(startOfLastWeek.year, int(currDate.split('/')[0]),int(currDate.split('/')[1]))
    if (startIndex == -1 and currDate == startOfLastWeek) :
        startIndex = i
    elif (startIndex != -1 and currDate > endOfLastWeek) :
        endIndex = i
        break
print("Starting Row:"  + str(startIndex))
print("Ending Row:"  + str(endIndex))

Starting Row:55845
Ending Row:61819


In [32]:
weekTable = checkout.take(range(startIndex,endIndex))
weekTable

Date,Time,Item Barcode,Quantity,Item Name,Remaining #
Monday 1/30,15:09,fruit,0,Fresh Fruit,500
,,vegetable,3,Fresh Veggies,497
,,spices,0,Spices,500
,,bread,1,Bread,299
,,potato,0,Potato,400
,,onion,0,Onion,0
,,027862020168,1,Soft Tofu,71
Monday 1/30,15:18,fruit,7,Fresh Fruit,493
,,vegetable,5,Fresh Veggies,492
,,spices,4,Spices,496


## Creating a table/graph with which day visitors came

In [33]:
dayTable = weekTable.group("Date").where("Date",are.not_equal_to(""))
dayTable



Date,count
Friday 2/3,88
Monday 1/30,100
Saturday 2/4,52
Thursday 2/2,224
Tuesday 1/31,80
Wednesday 2/1,149


In [34]:
dayTable.sort("count").barh("Date","count")
print("Total visitors: " + str(sum(dayTable.column("count"))))

Total visitors: 693


In [35]:
numHoursOpenPerDay = [2,2,1,5,2,3]
countPerHour = []
for i in range(len(numHoursOpenPerDay)):
    countPerHour.append(dayTable.column("count")[i]/numHoursOpenPerDay[i])

adjustedDayTable = dayTable.with_column("Count (per hour)", countPerHour)
adjustedDayTable


Date,count,Count (per hour)
Friday 2/3,88,44
Monday 1/30,100,50
Saturday 2/4,52,52
Thursday 2/2,224,44.8
Tuesday 1/31,80,40
Wednesday 2/1,149,49.6667


In [36]:
adjustedDayTable.sort("count").barh("Date")

## Creating a table/graph with item name and the number of times it was checked out

In [27]:
quantityTable = Table().with_columns("Item Name", weekTable.column("Item Name"), "Quantity", weekTable.column("Quantity")).group("Item Name", sum).sort("Quantity sum", descending = True)
quantityTable

Item Name,Quantity sum
Fresh Fruit,3073
Fresh Veggies,2079
"Totally Juice, 100% Berry Juice",1113
Potato,1017
Eggs,529
Onion,510
Canned Chicken,425
North Pride Whole Kernel Corn,377
Dairy Pure Lowfat Milk,372
Roberts White Rice,349


In [28]:
quantityTable.barh("Item Name", "Quantity sum")
print("Total number of items checked out: " + str(sum(quantityTable.column("Quantity sum"))))

Total number of items checked out: 13551


TODOS:

-Make date's customizable so that we can see any window of dates that we want

-Modify code so that it still works even if the start date is not in the checkout log (kind of like how the code handles not finding the end date in the code)

-Add other things you think would be useful stat info on weekly info update to the rest of the team (# of visitors, # of total items checked out, etc.) 

In [29]:
SPREADSHEET_ID = '1QDyC7uyM51JavfPHfmoY4DTBo3PoOAoN9CJwA4jhLuY'
RANGE_NAME = 'Check Out Form!A2:B'
timestampColumn = []
calIdColumn = []
def getSheetInfo():
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('sheets', 'v4', credentials=creds)

        # Call the Sheets API
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                    range=RANGE_NAME).execute()
        values = result.get('values', [])

        if not values:
            print('No data found.')
            return
        for row in values:
            if(7 <=len(row[1]) <= 14):
                timestampColumn.append(row[0])
                calIdColumn.append(row[1])
    except HttpError as err:
        print(err)

getSheetInfo()

In [ ]:
checkin = Table().with_columns("Timestamp", timestampColumn,"Cal Id", calIdColumn)
checkin

Timestamp,Cal Id
8/30/2022 12:02:32,810:00544319
8/30/2022 12:02:37,810:00544319
8/30/2022 12:02:39,810:00543961
8/30/2022 12:03:26,810:00574591
8/30/2022 12:03:41,810:00555155
8/30/2022 12:04:04,810:00554760
8/30/2022 12:04:11,810:00554760
8/30/2022 12:04:22,810:00529859
8/30/2022 12:04:30,810:00528996
8/30/2022 12:05:25,810:00514762


In [ ]:
checkinGroup = checkin.group("Cal Id").sort("count",descending=True)
checkinGroup = checkinGroup.relabeled("count","Number of Visits this AY")
checkinGroup.show(20)


Cal Id,Number of Visits this AY
810:00377345,35
810:00276152,32
810:00567863,31
810:00585326,30
810:00562630,24
810:00220667,23
810:00279298,23
810:00420882,23
810:00489115,23
810:00562635,23


In [ ]:
checkInCount = checkinGroup.group("Number of Visits this AY")
checkInCount.barh("Number of Visits this AY", "count")

In [ ]:
checkInCountWithoutOneTime = checkInCount.where("Number of Visits this AY", are.not_equal_to(1))
checkInCountWithoutOneTime.barh("Number of Visits this AY", "count")